### Remove DB

In [1]:
import os

if os.path.exists('example.db'):
    os.remove('example.db')

### Setup DB

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

from param_persist.sqlalchemy.models import Base


engine = create_engine('sqlite:///example.db')
Base.metadata.create_all(engine)

connection = engine.connect()
transaction = connection.begin()
session = Session(bind=connection)

### Param Class

In [3]:
import param
import numpy as np

class ExampleParamClass(param.Parameterized):
    number_field = param.Number(0.5, doc="A simple number field.")
    integer_field = param.Integer(1, doc="A simple integer field.")
    string_field = param.String("My String", doc="A simple string field.")
    bool_field = param.Boolean(False, doc="A simple boolean field.")
    

### Create Agent

In [4]:
from param_persist.agents.sqlalchemy_agent import SqlAlchemyAgent
from param_persist.sqlalchemy.models import InstanceModel, ParamModel

agent = SqlAlchemyAgent(engine=engine)

In [5]:
def show_database(db_session):
    print('Instances:')
    for parameterized_instance in db_session.query(InstanceModel).all():
        print(f'\t{parameterized_instance}')
    
    print('Parameters:')
    for parameter in db_session.query(ParamModel).all():
        print(f'\t{parameter}\n\t\t{parameter.value}')
    

### Save a Parameterized Object

In [6]:
show_database(session)

my_param = ExampleParamClass()
my_param_id = agent.save(my_param)

show_database(session)


Instances:
Parameters:
Instances:
	<Instance(id="b083a24f-f957-432c-a910-417cf1136cae", class_path="__main__.ExampleParamClass")>
Parameters:
	<Param(id="fa8205a6-a72c-4f10-9898-5b5864ef7399", instance_id="b083a24f-f957-432c-a910-417cf1136cae")>
		{"name": "bool_field", "value": false, "type": "param.Boolean"}
	<Param(id="d0a0dee4-e267-45fa-b151-9b1cfd7d6f6f", instance_id="b083a24f-f957-432c-a910-417cf1136cae")>
		{"name": "integer_field", "value": 1, "type": "param.Integer"}
	<Param(id="93bb5c1e-be6c-4369-bfb8-2d252b92508e", instance_id="b083a24f-f957-432c-a910-417cf1136cae")>
		{"name": "number_field", "value": 0.5, "type": "param.Number"}
	<Param(id="f7051e3e-75ea-45fa-ab03-ab4db51de235", instance_id="b083a24f-f957-432c-a910-417cf1136cae")>
		{"name": "string_field", "value": "My String", "type": "param.parameterized.String"}



### Update a Parameterized Object

In [7]:
my_param.number_field = 2.2
my_param.bool_field = True

agent.update(my_param, my_param_id)

show_database(session)

Instances:
	<Instance(id="b083a24f-f957-432c-a910-417cf1136cae", class_path="__main__.ExampleParamClass")>
Parameters:
	<Param(id="fa8205a6-a72c-4f10-9898-5b5864ef7399", instance_id="b083a24f-f957-432c-a910-417cf1136cae")>
		{"name": "bool_field", "value": true, "type": "param.Boolean"}
	<Param(id="d0a0dee4-e267-45fa-b151-9b1cfd7d6f6f", instance_id="b083a24f-f957-432c-a910-417cf1136cae")>
		{"name": "integer_field", "value": 1, "type": "param.Integer"}
	<Param(id="93bb5c1e-be6c-4369-bfb8-2d252b92508e", instance_id="b083a24f-f957-432c-a910-417cf1136cae")>
		{"name": "number_field", "value": 2.2, "type": "param.Number"}
	<Param(id="f7051e3e-75ea-45fa-ab03-ab4db51de235", instance_id="b083a24f-f957-432c-a910-417cf1136cae")>
		{"name": "string_field", "value": "My String", "type": "param.parameterized.String"}


### Load Parameterized Object

In [ ]:
my_param_loaded = agent.load(my_param_id)

print(my_param_loaded.bool_field)
print(my_param_loaded.number_field)
print(my_param_loaded.integer_field)
print(my_param_loaded.string_field)

### Delete Parameterized Object

In [ ]:
show_database(session)
agent.delete(my_param_id)
show_database(session)